In [4]:
import json
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords
from nltk.tag.perceptron import PerceptronTagger
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LinearRegression
import numpy as np 
import pickle
from sklearn.metrics import mean_squared_error
lemmatizer = WordNetLemmatizer()
tagger=PerceptronTagger()


In [5]:
with open("./training_set.json", "r") as f:
    data = f.read()
    train_data = json.loads(data)
    
print("number of training instances:", len(train_data))

with open("./test_set.json", "r") as f:
    data = f.read()
    test_data = json.loads(data)
print("number of testing instances:", len(test_data))

number of training instances: 1396
number of testing instances: 634


In [6]:
train_data[:5]

[{'sentiment': -0.463,
  'snippet': 'downgrade',
  'target': '$PCAR',
  'tweet': 'downgrades $SON $ARI $GG $FLTX $WMC $MFA $IVR $CMI $PCAR $QLIK $AFOP $UNFI #stocks #investing #tradeideas'},
 {'sentiment': 0.678,
  'snippet': ['looking sexy this morning', 'break on volume'],
  'target': '$AMZN',
  'tweet': "$AMZN looking sexy this morning...$600 break on volume and it's gone."},
 {'sentiment': 0.377,
  'snippet': 'still long term fan!',
  'target': '$SBUX',
  'tweet': "@GerberKawasaki stock hasn't moved much since first few weeks after split but still long term fan! $sbux"},
 {'sentiment': 0.129,
  'snippet': '$TFM will have a way to go price wise to compete with Kroger. $KR',
  'target': '$KR',
  'tweet': 'Whole foods $WFM may feel price competition but $TFM will have a way to go price wise to compete with Kroger. $KR https://t.co/XBxJVG94mx'},
 {'sentiment': 0.395,
  'snippet': 'iPhone SE Could Be Doing Better Than Expected',
  'target': '$AAPL',
  'tweet': "Apple's iPhone SE Could B

In [7]:
test_data[:5]

[{'sentiment': 0.323,
  'snippet': 'ooks pretty bullish for now',
  'target': '$ATVI',
  'tweet': "$ATVI ooks pretty bullish for now. from a short-term perspective, it's got a good chance of maybe sliding back to 33.70 #stocks #investing"},
 {'sentiment': 0.579,
  'snippet': ['looks really interesting on drop',
   'grabbed some options and stock',
   'enough tme before earnings to grow the stock',
   'growth all sectors'],
  'target': '$CSCO',
  'tweet': '$CSCO looks really interesting on drop, grabbed some options and stock, enough tme before earnings to grow the stock, growth all sectors'},
 {'sentiment': 0.294,
  'snippet': 'covered some shorts',
  'target': '$TSLA',
  'tweet': '$TSLA : covered some shorts @ 246.00 for +9.22pts'},
 {'sentiment': 0.028,
  'snippet': 'triple top forming.',
  'target': '$TSLA',
  'tweet': 'Watching $TSLA W triple top forming.'},
 {'sentiment': -0.076,
  'snippet': 'Whole Foods shareholders vote down activist initiatives',
  'target': '$WFM',
  'tweet':

In [8]:
# load NTUSD
with open("./NTUSD-Fin/NTUSD_Fin_word_v1.0.json", "r") as f:
    data = f.read()
    NTUSD = json.loads(data)

In [9]:
word_sent_dict = {}
for i in range(len(NTUSD)):
    word_sent_dict[NTUSD[i]["token"]] = NTUSD[i]["market_sentiment"]

In [14]:
stop_words = set(stopwords.words('english'))
stop_word= ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', '@', '#']
def remove_stopwords(data):
    sentence_token = [s.split(' ') for s in data] 
    idx = 0
    for sentence in sentence_token:
        clean_sentence_token = []
        for word in sentence:
            #if word not in list(stop_words):
            word= ''.join(c for c in word if c not in stop_word)
            if word != '':
                clean_sentence_token.append(word.lower())
        sentence_token[idx] = clean_sentence_token
        idx = idx + 1
    return sentence_token

In [15]:
train_X = [item["tweet"].lower() for item in train_data]
train_X = remove_stopwords(train_X)
train_y = np.array([item["sentiment"] for item in train_data],dtype=np.float)

test_X = [item["tweet"].lower() for item in test_data]
test_X = remove_stopwords(test_X)
test_y = np.array([item["sentiment"] for item in test_data],dtype=np.float)

In [51]:
def sentByDict(sent):
    sent = sent.split()
    sent_value = []
    for s in sent:
        try: 
            sent_value.append(word_sent_dict[s])
        except:
            pass
    if sent_value==[]:
        return 0
    else:
        return np.average(sent_value)
def get_wordnet_tag(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

def get_score(sentence):
    sent_value = []
    #print(sentence)
    sentence_tagged = np.array(nltk.pos_tag(sentence))
    #print(sentence_tagged)
    for tagged in sentence_tagged:
        wn_tag = get_wordnet_tag(tagged[1])
        word = tagged[0]
        
        nltk_sentiwordnet_score = 0.
        #get sentiwordnet score
        if wn_tag in (wn.NOUN, wn.ADJ, wn.ADV,  wn.VERB):            
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if lemma:
                synsets = wn.synsets(lemma, pos=wn_tag)
                if synsets:
                    swn_synset = swn.senti_synset(synsets[0].name())
                    nltk_sentiwordnet_score = swn_synset.pos_score() - swn_synset.neg_score()
    
        #get NTUSD dict score
        try: 
            dict_score = word_sent_dict[word]
        except:
            dict_score = 0.0
        
        word_score = np.array([nltk_sentiwordnet_score, dict_score], dtype=float)
        sent_value.append(word_score)
    #print(sent_value)
    return np.average(np.array(sent_value), axis=0)
    
    
    
    
    
train_pred = np.array([get_score(sentence) for sentence in train_X])
train_nor_pred = (((train_pred-np.min(train_pred,axis=0))/ (np.max(train_pred, axis=0)-np.min(train_pred,axis=0))) - 0.5)*2.

test_pred = np.array([get_score(sentence) for sentence in test_X])
test_nor_pred = (((test_pred-np.min(test_pred, axis=0))/ (np.max(test_pred, axis=0)-np.min(test_pred, axis=0))) - 0.5)*2.

In [52]:
test_nor_pred

array([[ 0.02378999,  0.56439723],
       [-0.11492281,  0.57141767],
       [-0.32075472,  0.34361474],
       ...,
       [-0.20754717,  0.56222534],
       [-0.37466307,  0.42168027],
       [-0.32075472,  0.47319065]])

In [11]:
print("train mse:", mean_squared_error(y_pred=train_nor_pred, y_true=train_y))
print("test mse:", mean_squared_error(y_pred=test_nor_pred, y_true=test_y))

train mse: 0.28521670923266274
test mse: 0.2614622213278173


In [93]:
# load RNN based predictoin
with open("rnn_pred.txt", "r") as f:
    rnn_test_pred = f.read().split("\n")
with open("rnn_train_pred.txt", "r") as f:
    rnn_train_pred = f.read().split("\n")

In [94]:
combine_train_pred = np.vstack((train_pred, np.array(rnn_train_pred))).astype(np.float).T
combine_test_pred = np.vstack((test_pred, np.array(rnn_test_pred))).astype(np.float).T

In [95]:
combine_train_pred.shape

(1396, 2)

In [53]:
# use linear regression for mapping
lr = LinearRegression()
lr.fit(train_pred, train_y)
test_lr_pred = lr.predict(test_pred)

In [57]:
lr.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': 1, 'normalize': False}

In [54]:
print("test mse:",mean_squared_error(test_lr_pred, test_y))

test mse: 0.11045906941504247


1. create feature
    * average word embedding 
    * average NTUSD word sentiment
2. XGBoost


In [74]:
len(train_X) + len(test_X)

2030

In [57]:
# output .txt for rnn model prediction
with open("fin_tweet.txt", "w") as f:
    f.write("id,text\n")
    for i, line in enumerate(test_X):
        line = line.replace("\n"," ")
        f.write(str(i))
        f.write(",")
        f.write(line)
        if i+1 != len(test_X):
            f.write("\n")

with open("fin_tweet_train.txt", "w") as f:
    f.write("id,text\n")
    for i, line in enumerate(train_X):
        line = line.replace("\n"," ")
        f.write(str(i))
        f.write(",")
        f.write(line)
        if i+1 != len(train_X):
            f.write("\n")